SET UP ENVIRONMENT

In [2]:
%pip install langchain-openai
%pip install langchain
%pip install titoken
%pip install langchain-text-splitters
%pip install pypdf
%pip install pdfminer.six



Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement titoken (from versions: none)
ERROR: No matching distribution found for titoken


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


SET OPENAI_API_KEY AS ENVIRONMENT VARIABLE

In [3]:
import os
with open('secrets/api_key', 'r') as file:
    api_key = file.read().strip()  # read the file content and remove any leading or trailing whitespace
os.environ['OPENAI_API_KEY'] = api_key

IMPORT MODEL AND SET UP HYPERPARAMETERS

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
llm.temperature

0.0

PDF LOADER

In [5]:
from langchain_community.document_loaders import PDFMinerLoader

loader = PDFMinerLoader("b295_slim.pdf")
documentPDF = loader.load()

chunks = len(documentPDF)
print(f"Number of chunks: {chunks}, size of chunks: {llm.get_num_tokens(documentPDF[0].page_content)}")

Number of chunks: 1, size of chunks: 67353


COBOL TEXT SPLITTER

In [6]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter
    )
from langchain_core.documents.base import Document
cobol_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.COBOL, chunk_size = 10000, chunk_overlap = 1000)

cobol_docs = cobol_splitter.split_documents(documentPDF)
#cobol_docs = [Document(page_content=cobol_docs_original[0].page_content, metadeta=cobol_docs_original[0].metadata),
#               Document(page_content=cobol_docs_original[1].page_content, metadeta=cobol_docs_original[1].metadata),
#               Document(page_content=cobol_docs_original[2].page_content, metadeta=cobol_docs_original[2].metadata),
#               Document(page_content=cobol_docs_original[3].page_content, metadeta=cobol_docs_original[3].metadata),
#               Document(page_content=cobol_docs_original[4].page_content, metadeta=cobol_docs_original[4].metadata),
#               Document(page_content=cobol_docs_original[5].page_content, metadeta=cobol_docs_original[5].metadata)]

page = 0
for i, doc in enumerate(cobol_docs):
    if "PROCEDURE DIVISION" in cobol_docs[i].page_content:
        page = i
        break

procedure_division = cobol_docs[page:]

chunks = len(procedure_division)
print(f"Number of chunks: {chunks}, size of chunks: {llm.get_num_tokens(procedure_division[0].page_content)}")

Number of chunks: 19, size of chunks: 2715


SET UP PROMPTS FOR REFINE TECHNIQUES

In [8]:
from langchain.chains import RefineDocumentsChain, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI

# This controls how each document will be formatted. Specifically,
# it will be passed to `format_document` - see that function for more
# details.
document_prompt = PromptTemplate(
    input_variables=["page_content"],
     template="{page_content}"
)
document_variable_name = "context"

# The prompt here should take as an input variable the
# `document_variable_name`
prompt = PromptTemplate.from_template(
    #"Summarize the following COBOL code and explain its content to someone who isn't familiar with COBOL : {context}\n"
    "The following is the Cobol code of PROGRAM-ID B295 :\n {context} \n"
    ""
)

initial_llm_chain = LLMChain(llm=llm, prompt=prompt)
initial_response_name = "prev_response"
# The prompt here should take as an input variable the
# `document_variable_name` as well as `initial_response_name`
prompt_refine = PromptTemplate.from_template(
    #"Here's your first summary: of a cobol program: {prev_response}.\n"
    #"Now add to it based on the content: {context}\n"
    #"Explain its content to someone who isn't familiar with COBOL"
    "Here's info about the Cobol program PROGRAM-ID B295: {prev_response}.\n"
    "Provide a conci: {context}\n"
)
refine_llm_chain = LLMChain(llm=llm, prompt=prompt_refine)

chain = RefineDocumentsChain(
    initial_llm_chain=initial_llm_chain,
    refine_llm_chain=refine_llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
    initial_response_name=initial_response_name,
    return_intermediate_steps = True
)



RUN THE CHAIN

In [9]:
print(chain.invoke(procedure_division))

{'input_documents': [Document(page_content="*                  , CEMTIT \n\n*                  , TSALCTIT \n\n*                  , TBLACCST \n\n*                  , TBLPREN \n\n*                  , TBLINMT \n\n*                  , TBLASS \n\n*                  , DECGIOR \n\n*                  , PECGIOR \n\n\x0c*                  , PFOECG \n\n*                  , TECGIOR \n\n*                  , DECPER \n\n*                  , PECPER \n\n*                  , PFOECP \n\n*                  , TECPER \n\n*                  , TSALPRES \n\n*              FROM  CET           T1 \n\n*             WHERE  CTCTIT = :CETY0.CTCTIT \n\n*                    AND CSOGG = :CETY0.CSOGG \n\n*                    AND CNAZTIT = :CETY0.CNAZTIT \n\n*                    AND CSPTIT = :CETY0.CSPTIT \n\n*                    AND CSUFF = :CETY0.CSUFF \n\n*                    AND CEMTIT = :CETY0.CEMTIT \n\n*    END-EXEC. \n\n*01  CTR-W002P01     PIC 9(1) VALUE 0. \n\n*    EXEC SQL DECLARE CUR-W002P01      CURSOR FOR \